In [365]:
import numpy as np
import pyaudio
import time
from pyldpc import make_ldpc, encode, decode, get_message

# Ses kayıt parametreleri
CHUNK = 1024*2
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100

# Taşıyıcı frekansı
tasiyici_frekans = 3000

# LDPC kodlama parametreleri
n = 32
d_v = 4
d_c = 8
snr = 1000
H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
k = G.shape[1]

# PyAudio nesnesi oluştur
p = pyaudio.PyAudio()

# Çıkış akışını aç
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    output=True,
    frames_per_buffer=CHUNK
)

# Gönderilecek veri (binary)
v = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1], dtype=np.int16)
y = encode(G, v, snr)

# Ses örneği oluşturma
sample_rate = 44100
duration = 1  # Sesin süresi (saniye)
samples = int(sample_rate * duration)
time_samples = np.arange(samples) / sample_rate
sound_wave = np.sin(2 * np.pi * 440 * time_samples)  # 440 Hz'lik bir sinüs dalgası

# Ses verisini bitlere dönüştürme
sound_bits = np.unpackbits(sound_wave.astype(np.int16).view(np.uint8))

# Bitleri kodlama
encoded_bits = encode(G, sound_bits, snr)

# AM Modülasyonu
t = np.arange(0, len(encoded_bits)) / RATE
tasiyici_dalga = np.sin(2 * np.pi * tasiyici_frekans * t)
modüle_edilmis_dalga = tasiyici_dalga * (encoded_bits * 2 - 1)

# Modüle edilmiş dalgayı ses cihazına gönderme
for i in range(50):
    stream.write(modüle_edilmis_dalga.astype(np.int16).tobytes())
    time.sleep(0.0002)

# Akışı kapat
stream.stop_stream()
stream.close()
p.terminate()

ValueError: shapes (32,19) and (705600,) not aligned: 19 (dim 1) != 705600 (dim 0)

In [367]:
print(len(sound_bits))


705600


In [198]:
>>> import numpy as np
>>> from pyldpc import make_ldpc, encode, decode, get_message
>>> n = 32
>>> d_v = 4
>>> d_c = 8
>>> snr = 1000
>>> H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
>>> k = G.shape[1]
>>> v = np.random.randint(2, size=k)
>>> y = encode(G, v, snr)
>>> d = decode(H, y, snr)
>>> x = get_message(G, d)
>>> assert abs(x - v).sum() == 0

In [199]:
print(len(y))

32


In [200]:
print(len(v))

19


In [197]:
k

1027

In [59]:
d = decode(H, y, 20)
x = get_message(G, d)
x


array([1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0])

In [179]:
print(len(y))

2048


In [229]:
y

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1.])

In [208]:
m

array([-0.        , -4.02379232, -1.1710782 , ...,  4.19242495,
        1.43320713, -3.7753066 ])

In [206]:
aci = np.arcsin(0.5)  

In [239]:
modüle_edilmis_dalga

array([    -0.        , -13958.18344955, -27633.50355162, ...,
       -70857.59588664, -60464.94083526, -48846.96046613])

In [296]:
m2=modüle_edilmis_dalga/32767

In [297]:
m2

array([-0.        , -0.42598295, -0.84333334, ..., -2.16246821,
       -1.84529987, -1.49073643])

In [330]:
t = np.arange(0, 2048) / RATE
tasiyici_dalga = np.sin(2 * np.pi * 1000 * t)
tasiyici_dalga=np.where(tasiyici_dalga==0,1e-10,tasiyici_dalga)




In [326]:
genis_veri=(m2/tasiyici_dalga +1)/2


In [327]:
al(genis_veri)

array([1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [328]:
def al(data):
	n = 32
	d_v = 4
	d_c = 8
	snr = 20
	H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
	gruplar = [data[i:i+64] for i in range(0, len(data), 64)]
	ana_ort=np.abs(np.mean(data))
	ortalamalar =[]
	
	for grup in gruplar:
		ortalama=np.mean(grup)
		if ortalama <0:
			ortalamalar.append(ortalama)
		else:
			ortalamalar.append(ortalama)
	ortalamalar=np.array(ortalamalar)	
	#print(ortalamalar[0])	
	d = decode(H, ortalamalar, snr)
	x=get_message(G, d)
			
	return x

In [329]:
genis_veri

array([ 0.5, -1. , -1. , ..., -1. , -1. , -1. ])

In [248]:
1e-10*10**10


1.0

In [255]:
genis_veri

array([ 0.5       ,  0.28471693, -0.96442318, ...,  1.60231922,
        3.25157234, -0.34386229])

In [256]:
genisletilmis_veri

array([-1., -1., -1., ..., -1., -1., -1.])